# Hackathon Dotz

##### Classificar as observações por (SUB-CATEGORIA | CATEGORIA | DEPARTAMENTO)

In [1]:
# Importação dos pacotes para Analise
 
import pandas as pd
import seaborn as sns
import pprint
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
from google.colab import files  
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')

import warnings
warnings.simplefilter(action = 'ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
#Carregar os dados para o google colab

uploaded = files.upload()

Saving Hackathon_Base_Teste.csv to Hackathon_Base_Teste.csv
Saving Hackathon_Base_Treino_comdep.csv to Hackathon_Base_Treino_comdep.csv


In [0]:
# Importação dos dataset
df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv('Hackathon_Base_Teste.csv')

### Análise dos dados

In [0]:
# Dicionario de balanceamento SUB-CATEGORIA

sub_ = df['SUB-CATEGORIA'].value_counts() == 1
sub_ = sub_[sub_.isin([True])]
lista_sub_ = list(sub_.index)
lista_sub_

['COM WHEY',
 'SUÍNA APIMENTADA',
 'BÁSICA EEG',
 'CARIOCA',
 'VEÍCULOS ACESSÓRIOS',
 'TAPIOQUEIRA',
 'ADESIVOS REFIL',
 'FUSILLI SEM GLÚTEN',
 'ASPARGO',
 'GRÃOS',
 'CAPPUCCINO LIGHT',
 'FRALDA GERIATRICA P',
 'AÇÚCAR COLORIDO',
 'SOPA CLARA CARNE',
 'SALADA DE ATUM',
 'ANTI-COCEIRA',
 'COLA LIGHT',
 'CARTÃO DE MEMÓRIA',
 'PALHA ITALIANA',
 'ZERO UVA',
 'MEXILHÃO',
 'SERRA',
 'CORNETINHA',
 'FRANGO COM CATUPIRY',
 'ACESSÓRIOS MANICURE',
 'SELETA DE LEGUMES',
 'MOLHO IOGURTE',
 'FROST FREE 2 PORTAS',
 'GRAXA',
 'BLEND',
 'PENDRIVE',
 'FRESH',
 'PORTA CHUPETA',
 'LAVA AUTO TRADICIONAL',
 'CHUVEIRINHO TORNEIRA',
 'BALANÇA',
 'MOUSE COM FIO',
 'PÓ/LÍQUIDO',
 'QUEIJADILHA',
 'SAL LIGHT',
 'LENHA',
 'BOMBA CHIMARRÃO',
 'SACO PARA CONFEITAR',
 'NATURAL SEMI DESNATADO',
 'LAVANDA',
 'CONJUNTO PARA QUEIJO',
 'PANO ALTA PERFORMANCE',
 'PIRULITO SORTIDO',
 'SACOLA RETORNÁVEL',
 'PRATINHO',
 'MOSTARDA E MEL',
 'CHIPOTLE',
 'ÓLEO DE LINHAÇA',
 'CESTO',
 'COCADA',
 'PREMIUM M',
 'MINI BOMBOM',
 'PO

In [0]:
# Duplicando SUB-CATEGORIAS com apenas 1 produto
df_aux = df.rename(columns={'SUB-CATEGORIA': 'SUBCATEGORIA'})
df_aux = df_aux.query('SUBCATEGORIA in (@lista_sub_)')
df_aux.rename(columns={'SUBCATEGORIA': 'SUB-CATEGORIA'}, inplace=True)
df = df.append(df_aux)

In [0]:
df_aux = df.rename(columns={'SUB-CATEGORIA': 'SUBCATEGORIA'})
df_aux = df_aux.query('SUBCATEGORIA in ("TRADICIONAL")')
df_aux

,DESCRIÇÃO PARCEIRO,SUBCATEGORIA,CATEGORIA,DEPARTAMENTO
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE
67,CONDICIONADOR 1L BIOEXTRATUS TUTANO,TRADICIONAL,CONDICIONADOR,CUIDADOS COM O CABELO
71,LEITE PO NINHO INST L400GP360G,TRADICIONAL,LEITE EM PÓ,MERCEARIA SALGADA
104,AMAC ROUPA QBOA MACIEZ C/ CARINHO 500ML,TRADICIONAL,AMACIANTE,CUIDADOS COM A ROUPA
127,CREME DE LEITE BETANIA 200G,TRADICIONAL,CREME DE LEITE,MERCEARIA DOCE
...,...,...,...,...
21856,MOLHO SHOYO SAKURA TRAD PET 500ML,TRADICIONAL,MOLHO SHOYU,MERCEARIA SALGADA
21930,COND SKALA 325ML BOMBA ABACATE,TRADICIONAL,CONDICIONADOR,CUIDADOS COM O CABELO
21934,LEITE FERMENTADO TREVINHO C/6 480G,TRADICIONAL,LEITE FERMENTADO,IOGURTES
21993,YAKULT 40 LEITE FERMENTADO 80g,TRADICIONAL,LEITE FERMENTADO,IOGURTES


### Etapa de Preparação do Dados

In [0]:
# Variáveis importantes
pontuacao = ['.',',','-','+',':',';','&','+','/','!','#','%','(',')','  ']
# Palavras para retirar da análise
stop_words = ['em','sao','ao','de','da','do','para','c','kg','un','ml','pct','und','das','no','ou', #GERAL
              'com','sem','gfa','jg','la','1','2','3','4','5','6','7','8','9','0','a','b','c','d','e', #GERAL
              'f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','x','w','y','z']
stop_words_sub = ['em','sao','ao','de','da','do','com','para','c','kg','un','ml','pct','und','das',
                  'no','ou','sem','gfa','jg','la','1','2','3','4','5','6','7','8','9','0']

# Tamanho da validação de teste
test_size = 0.3
random_state = 42
# Parametros dos modelos NB
model = MultinomialNB
alpha = 0.1
fit_prior = False
# Parametros do vetor CountVectorizer
ngram_range = (1, 1)
strip_accents = 'ascii'
# Parametros do vetor TfidfTransformer
use_idf = True

In [13]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)
df['DESC_AJUSTADA_SUB'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)
# Dados de Teste
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)
df2['DESC_AJUSTADA_SUB'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')
  df['DESC_AJUSTADA_SUB'] = df['DESC_AJUSTADA_SUB'].str.replace(x, ' ')
  # Dados de Teste
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')
  df2['DESC_AJUSTADA_SUB'] = df2['DESC_AJUSTADA_SUB'].str.replace(x, ' ')
  
df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA,DESC_AJUSTADA_SUB
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,PASTA INT VITAPOWER KG AMEND SHOT,PASTA INT VITAPOWER KG AMEND SHOT
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C,ESPONJA BETTANIN BRILHUS C


In [0]:
'''
# Função para retirar stop words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stop_words]
  new_text = TreebankWordDetokenizer().detokenize(stop)
  return new_text

#Ajustando os departamentos
df['DEP_AJUSTADA'] = df['DEPARTAMENTO'].str.lower()

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
for x in pontuacao:
  df['DEP_AJUSTADA'] = df['DEP_AJUSTADA'].str.replace(x, ' ')

# Eliminando as stop words
df['DEP_AJUSTADA'] = df['DEP_AJUSTADA'].apply(tokenize_df)
df.head()
'''

"\n# Função para retirar stop words\ndef tokenize_df(tokenized_words):\n  tokenized_words = word_tokenize(tokenized_words)\n  stop = [word for word in tokenized_words if word not in stop_words]\n  new_text = TreebankWordDetokenizer().detokenize(stop)\n  return new_text\n\n#Ajustando os departamentos\ndf['DEP_AJUSTADA'] = df['DEPARTAMENTO'].str.lower()\n\n# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.\nfor x in pontuacao:\n  df['DEP_AJUSTADA'] = df['DEP_AJUSTADA'].str.replace(x, ' ')\n\n# Eliminando as stop words\ndf['DEP_AJUSTADA'] = df['DEP_AJUSTADA'].apply(tokenize_df)\ndf.head()\n"

### Etapa 1 - Classificando DEPARTAMENTO

In [14]:
#Criando a nova para classificar DEPARTAMENTO
base = df
 
# Selecionando apenas o item a ser classificado e o target do DF principal
base_data = base['DESC_AJUSTADA']
base_target = base['DEPARTAMENTO']

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(base_data, base_target, test_size=test_size, random_state=random_state)

# Montando Pipeline
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=ngram_range, strip_accents=strip_accents, stop_words=stop_words)),
                     ('tfidf', TfidfTransformer(use_idf=use_idf)),
                     ('clf', SVC(gamma='scale', kernel='linear', shrinking=True, tol=0.01)) ])
                     #('clf', model(alpha=alpha, fit_prior=fit_prior)) ])
 
text_clf = text_clf.fit(x_train, y_train)

# Avaliando a performance com predição
predicted = text_clf.predict(x_test)
 
print('\n###Indicadores Classificação###\n')
print(metrics.classification_report(y_test, predicted))


###Indicadores Classificação###

                          precision    recall  f1-score   support

              AUTOMOTIVO       0.97      0.86      0.91        80
                 AÇOUGUE       0.96      0.89      0.93       147
      BEBIDAS ALCOÓLICAS       0.98      0.97      0.98       407
  BEBIDAS NÃO ALCOÓLICAS       0.96      0.96      0.96       224
               BRINQUEDO       0.95      0.74      0.83        53
                 CALÇADO       1.00      1.00      1.00       189
      CAMA, MESA E BANHO       0.98      0.99      0.98       127
        CONGELADOS DOCES       0.92      0.94      0.93        83
     CONGELADOS SALGADOS       0.83      0.74      0.78        34
    CUIDADOS COM A BARBA       1.00      0.95      0.98        21
     CUIDADOS COM A CASA       0.88      0.91      0.89       144
  CUIDADOS COM A COZINHA       0.82      0.71      0.76        38
    CUIDADOS COM A ROUPA       0.94      0.99      0.96        93
 CUIDADOS COM MÃOS E PÉS       0.96      

In [19]:

# APENAS AVALIAÇÃO PERFORMANCE PREDIÇÃO DEPARTAMENTO

def performance(valor):
  # Montando Pipeline
  text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', valor()) ])
  text_clf = text_clf.fit(x_train, y_train)

  # Valindando performance
  predicted = text_clf.predict(x_test)
  print('\n', valor)
  print('Performance com os paramentros default\n', np.mean(predicted == y_test))

  # Melhores paramentros
  parameters = {#'vect__ngram_range': [(1, 1), (1, 2)],
                'clf__max_iter': (5000, 10000, 15000),
                'clf__validation_fraction': (0.01, 0.001, 0.0001),
                }

  gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
  gs_clf = gs_clf.fit(x_train, y_train)

  print('Melhor score\n', gs_clf.best_score_)
  print('Melhores parametros')
  pprint.pprint(gs_clf.best_params_) 
  print('=' * 80)
  return True

models = ['BernoulliNB', 'ComplementNB', 'MultinomialNB']

performance(PassiveAggressiveClassifier)
#performance(BernoulliNB)
#performance(ComplementNB)
#performance(MultinomialNB)



 <class 'sklearn.linear_model._passive_aggressive.PassiveAggressiveClassifier'>
Performance com os paramentros default
 0.9407844918976223
Melhor score
 0.9414516048519943
Melhores parametros
{'clf__max_iter': 10000, 'clf__validation_fraction': 0.0001}


True

In [0]:
# Predição em novos dados com o DF  de teste
base_novos = df2['DESC_AJUSTADA']

#Previsão dos novos dados
predicted_novos = text_clf.predict(base_novos)

# Adicionando a coluna para o novo DF
df_dep = df2[['DESCRIÇÃO PARCEIRO','DESC_AJUSTADA']]
df_dep['DEPARTAMENTO'] = predicted_novos
df_dep['DESC_AJUSTADA_SUB'] = df2['DESC_AJUSTADA_SUB']

# Novos Dados de teste incluindo o departamento
df_dep.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO G,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C DR PR BC R,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R


In [0]:
'''
# Gerando dataframe para validação
def detokenize(texto):
  novo_texto = TreebankWordDetokenizer().detokenize(texto)
  return novo_texto

valida_dep = pd.DataFrame(data=y_test, columns=['DEPARTAMENTO'])
valida_dep['DEPARTAMENTO_PREDIÇÃO'] = predicted
valida_dep['DESCRIÇÃO'] = vetor.inverse_transform(x_test)
valida_dep['DESCRIÇÃO'] = valida_dep['DESCRIÇÃO'].apply(detokenize)
valida_dep['VALIDA'] = valida_dep['DEPARTAMENTO_PREDIÇÃO'] == valida_dep['DEPARTAMENTO']
valida_dep = valida_dep.query('VALIDA == False')

valida_departamento = valida_dep['DESCRIÇÃO'] + '     |' + valida_dep['DEPARTAMENTO'] + '    |' + valida_dep['DEPARTAMENTO_PREDIÇÃO']
valida_departamento = str(sorted(list(valida_departamento)))

# Gerar um arquivo com a lista de palavras unicas
arquivo = open('valida_departamento.txt', 'w')
arquivo.write(str(valida_departamento))
arquivo.close()
'''

"\n# Gerando dataframe para validação\ndef detokenize(texto):\n  novo_texto = TreebankWordDetokenizer().detokenize(texto)\n  return novo_texto\n\nvalida_dep = pd.DataFrame(data=y_test, columns=['DEPARTAMENTO'])\nvalida_dep['DEPARTAMENTO_PREDIÇÃO'] = predicted\nvalida_dep['DESCRIÇÃO'] = vetor.inverse_transform(x_test)\nvalida_dep['DESCRIÇÃO'] = valida_dep['DESCRIÇÃO'].apply(detokenize)\nvalida_dep['VALIDA'] = valida_dep['DEPARTAMENTO_PREDIÇÃO'] == valida_dep['DEPARTAMENTO']\nvalida_dep = valida_dep.query('VALIDA == False')\n\nvalida_departamento = valida_dep['DESCRIÇÃO'] + '     |' + valida_dep['DEPARTAMENTO'] + '    |' + valida_dep['DEPARTAMENTO_PREDIÇÃO']\nvalida_departamento = str(sorted(list(valida_departamento)))\n\n# Gerar um arquivo com a lista de palavras unicas\narquivo = open('valida_departamento.txt', 'w')\narquivo.write(str(valida_departamento))\narquivo.close()\n"

### Etapa 2 - Classificando CATEGORIA

In [0]:
#Criando a nova para classificar CATEGORIA
base = df
 
# Selecionando apenas o item a ser classificado e o target do DF principal
base_data = base['DEPARTAMENTO'] + ' ' + base['DESC_AJUSTADA']
base_target = base['CATEGORIA']

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(base_data, base_target, test_size=test_size, random_state=random_state)

# Executando Pipeline
text_clf = text_clf.fit(x_train, y_train)

# Avaliando a performance com predição
predicted = text_clf.predict(x_test)
 
print('\n###Indicadores Classificação###\n')
print(metrics.classification_report(y_test, predicted))


###Indicadores Classificação###

                                  precision    recall  f1-score   support

             ACESSÓRIOS DE BANHO       1.00      1.00      1.00         1
               ACESSÓRIOS ELETRO       1.00      1.00      1.00         2
                  ACESSÓRIOS PET       1.00      1.00      1.00         2
              AGUARDENTE/CACHAÇA       1.00      0.50      0.67         2
                         ALGODÃO       1.00      1.00      1.00         1
              ALIMENTO PARA CÃES       0.00      0.00      0.00         0
             ALIMENTO PARA GATOS       0.00      0.00      0.00         1
       ANTISSÉPTICO BUCAL ADULTO       1.00      1.00      1.00         2
                           ARROZ       1.00      1.00      1.00         1
             ARTIGOS PARA FESTAS       1.00      0.80      0.89         5
              AUTOMOTIVO FILTROS       1.00      1.00      1.00         1
                   AVE CONGELADA       1.00      1.00      1.00         1
   

In [0]:
'''
# APENAS AVALIAÇÃO PERFORMANCE PREDIÇÃO CATEGORIA

# Montando Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', PassiveAggressiveClassifier()) ])
text_clf = text_clf.fit(x_train, y_train)

# Valindando performance
predicted = text_clf.predict(x_test)
print('Performance com os paramentros default\n', np.mean(predicted == y_test))

# Melhores paramentros
parameters = {'vect__ngram_range': [(1, 2), (1, 3)],
              'vect__strip_accents': ('unicode', 'ascii'),
              'tfidf__use_idf': (True, False),
              'clf__tol': (0.01, 0.001),
              'clf__max_iter': (2000, 3000), }

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(x_train, y_train)

print('Melhor score\n', gs_clf.best_score_)
print('Melhores parametros')
pprint.pprint(gs_clf.best_params_)  
'''

"\n# APENAS AVALIAÇÃO PERFORMANCE PREDIÇÃO CATEGORIA\n\n# Montando Pipeline\ntext_clf = Pipeline([('vect', CountVectorizer()),\n                     ('tfidf', TfidfTransformer()),\n                     ('clf', PassiveAggressiveClassifier()) ])\ntext_clf = text_clf.fit(x_train, y_train)\n\n# Valindando performance\npredicted = text_clf.predict(x_test)\nprint('Performance com os paramentros default\n', np.mean(predicted == y_test))\n\n# Melhores paramentros\nparameters = {'vect__ngram_range': [(1, 2), (1, 3)],\n              'vect__strip_accents': ('unicode', 'ascii'),\n              'tfidf__use_idf': (True, False),\n              'clf__tol': (0.01, 0.001),\n              'clf__max_iter': (2000, 3000), }\n\ngs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)\ngs_clf = gs_clf.fit(x_train, y_train)\n\nprint('Melhor score\n', gs_clf.best_score_)\nprint('Melhores parametros')\npprint.pprint(gs_clf.best_params_)  \n"

In [0]:
# Predição em novos dados utilizando o df_dep
base_novos = df_dep['DEPARTAMENTO'] + ' ' + df_dep['DESC_AJUSTADA']

#Previsão dos novos dados
predicted_novos = text_clf.predict(base_novos)

# Adicionando a coluna no novo DF
df_cat = df_dep
df_cat['CATEGORIA'] = predicted_novos
df_cat['DESC_AJUSTADA_SUB'] = df_dep['DESC_AJUSTADA_SUB']

# Novos Dados de teste incluindo o departamento
df_cat.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB,CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO G,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G,SAL
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C DR PR BC R,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R,ENFEITES DE NATAL


### Etapa 3 - Classificando SUB-CATEGORIA

In [0]:
# Criando a nova para classificar SUB-CATEGORIA retirando apenas números
base = df

# Selecionando apenas o item a ser classificado e o target do DF principal
base_data = base['DEPARTAMENTO'] + ' ' + base['CATEGORIA'] + ' ' + base['DESC_AJUSTADA_SUB']
base_target = base['SUB-CATEGORIA']

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(base_data, base_target, test_size=test_size, random_state=random_state)

# Executando Pipeline
text_clf = text_clf.fit(x_train, y_train)

# Avaliando a performance com predição
predicted = text_clf.predict(x_test)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test, predicted))


###Indicadores Classificação Categoria###

                                precision    recall  f1-score   support

                       ABÓBORA       1.00      1.00      1.00         1
            ACESSÓRIOS CELULAR       0.00      0.00      0.00         0
              ACESÓRIOS PANELA       1.00      1.00      1.00         1
           AEROSOL COM PERFUME       1.00      1.00      1.00         2
                        ALFACE       0.00      0.00      0.00         0
               ALFACE ORGÂNICO       0.00      0.00      0.00         1
                    ALHO PASTA       1.00      1.00      1.00         1
              ALHO TRADICIONAL       1.00      1.00      1.00         1
          ALMOFADA TRADICIONAL       0.00      0.00      0.00         0
                      ALVEJADO       0.00      0.00      0.00         1
                      AMENDOIM       1.00      1.00      1.00         1
      ANTISSÉPTICO BUCAL MENTA       1.00      1.00      1.00         1
ANTISSÉPTICO BUCAL 

In [0]:
# Predição em novos dados
base_novos = df_cat['DEPARTAMENTO'] + ' ' + df_cat['CATEGORIA'] + ' ' + df_cat['DESC_AJUSTADA_SUB']

#Previsão dos novos dados
predicted_novos = text_clf.predict(base_novos)

# Adicionando a coluna
df_sub = df_cat
df_sub['SUB-CATEGORIA'] = predicted_novos

# Novos Dados de teste incluindo o departamento
df_sub.head(10)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB,CATEGORIA,SUB-CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO G,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G,SAL,SAL DO HIMALAIA
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C DR PR BC R,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R,ENFEITES DE NATAL,ENFEITES DE ÁRVORE DE NATAL
2,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA ML CX LIMAO,MERCEARIA DOCE,MIST PREP LIQ CHA VDE LINEA ML CX LIMAO,CHÁ,FRUTAS
3,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,CUIDADOS COM MÃOS E PÉS,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,ACESSÓRIOS MÃO E PÉ,ALICATE
4,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN,PRATOS PRONTOS,MEIA PIZZA PEPPERONI UN,PIZZA,MIX
5,JARRA ACASA VIDRO CURVES OJG378,JARRA ACASA VIDRO CURVES OJG,UTILIDADES DE COZINHA,JARRA ACASA VIDRO CURVES OJG,UTENSÍLIOS DE VIDRO,JARRA/GARRAFA
6,FILME DE PVC CONSERV PACK 30M,FILME DE PVC CONSERV PACK M,DESCARTÁVEIS,FILME DE PVC CONSERV PACK M,FILME PVC,FILME PVC
7,CESTO VIME CRAFTWORK S-25X15XH13,CESTO VIME CRAFTWORK S XXH,LAVANDERIA,CESTO VIME CRAFTWORK S XXH,ORGANIZADOR,CESTO DE ROUPA
8,MAC BORTOLINI 500GR OVOS ESPAGUETE .,MAC BORTOLINI GR OVOS ESPAGUETE,MERCEARIA SALGADA,MAC BORTOLINI GR OVOS ESPAGUETE,MASSA SECA,ESPAGUETE
9,COND NOVEX OLEO OJON 300ML,COND NOVEX OLEO OJON ML,CUIDADOS COM O CABELO,COND NOVEX OLEO OJON ML,CONDICIONADOR,TRADICIONAL


In [0]:
dotz = df_sub[['DESCRIÇÃO PARCEIRO', 'SUB-CATEGORIA', 'CATEGORIA']]
dotz.to_csv('dotz.csv')

In [0]:
df.head()

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA,DESC_AJUSTADA_SUB
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,PASTA INT VITAPOWER KG AMEND SHOT,PASTA INT VITAPOWER KG AMEND SHOT
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C,ESPONJA BETTANIN BRILHUS C
2,AGUA MIN SCHIN S/GAS 500ML,SEM GÁS,ÁGUA MINERAL,BEBIDAS NÃO ALCOÓLICAS,AGUA MIN SCHIN S GAS ML,AGUA MIN SCHIN S GAS ML
3,FITA DUPLA FACE C/SUPORTE SCOTCH,FITA ADESIVA,PAPELARIA,PAPELARIA,FITA DUPLA FACE C SUPORTE SCOTCH,FITA DUPLA FACE C SUPORTE SCOTCH
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,PIZZA REGULAR,MASSA FRESCA,MASSAS FRESCAS,MASSA PIZZA ROMANHA OREGANO PCT G,MASSA PIZZA ROMANHA OREGANO PCT G


In [0]:
'''
pontuacao = ['.',',','-','+',':',';','&','+','/','!','#','%','(',')','  ']
stop_words = ['em','sao','ao','de','da','do','para','c','kg','un','ml','pct','und','das',
              'no','ou','sem','gfa','jg','p/','la','1','2','3','4','5','6','7','8','9','0','a',
              'b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v',
              'x','w','y','z','kit','po','liq','cr','sh','pt','des','imp','lt','vd','az','esm',
              'cx','bc','lv','ad','gf','mix','rf','fr','bj','int','cd','cz','tp','ap','rs','caixa',
              'sao','bom','esp','extra','premium','pto','uso','sol','garrafa','brasil','pc','conj',
              'ph','gold','cj','mais','cor','facil','ice','classic','br','vm','tt','em','far','fix',
              'sc','dr','aco','nadir','romana','bauducco','original','pe','peso','pro','sl',
              'und','seda','tradicional','arcor','teka','bas','dec','fina','palmolive','neutro',
              'sec','peq','baw','brinox','ext','fd','up','waw','seara','sadia','del','oetker',
              'royal','ret','boni','black','ln','new','preta','corttex','blue','big','ou','co','ex',
              'paf','pif','box','bdj','bestway','cinza','dourada','gluten','plasvale','protex','ype',
              'fs','for','no','das','lg','me','pvc','refil','tirolez','sm','friboi','ipa','pr','st',
              'xg','xxg','mc','nr','cp','ox','pa','pq','me','unid','ana','bi','di','gr',
              'hereford','eg','daterra','grill','apim','distriboi','peca','tras','real','temp', #AÇOUGUE
              'tatu','elgolli','bassi','ancho','korin','organico','copacol','cord','simunovic', #AÇOUGUE
              'montana','aurora','vacuo','francezinha','germania','rainha','gourmet','gessner', #AÇOUGUE
              'perdigão','natupeixe','ferreira','ceratti','alegra','top','quality','dochef','atm', #AÇOUGUE
              'band','argus','halal','juliatto','corella','pamplona','promocao','peça','cubos', #AÇOUGUE
              'superfrango','regina','tulipa','millenium','angus','las','piedras','borb','brachola', #AÇOUGUE
              'swift','beff','paulista','osso','temperada','temperado','rech','palha','borboleta', #AÇOUGUE
              'capa','paz','pel','res','touc','beleco','organica','best','beef','granel','pele', #AÇOUGUE
              'maturatta','chorizo','nabrasa','dagranja','bd','macedo','sabor','chef','soberano', #AÇOUGUE
              'defumada','coz','porcionado','sulita','minga','lar','valesul','frxb','levida','agreco', #AÇOUGUE
              'desos','jagua','pdc','entrec','rica','nobre','avestruz','viva','reserva','parmes','iqf', #AÇOUGUE
              'el','golli','ivaipora','nho','bento','nova','gonzales','oriental','nat','bja','pamplona', #AÇOUGUE
              'ouro','temperatto','especial','joaquim','frescal','def','sulita','importada','importado', #AÇOUGUE
              'choice','usa','carinhas','bem','coated','canoa','cain','falani','forno','minas','noisett',
              'rus','serya','rusti','sticks','zuppa','crinkle','falani','fontini','fries','lutosa','mccain',
              'noisettes','saute','eleve','carone','demarchi','floretes','daucy','empanitos','auroggets',
              'petisco','perdigao','leoni','pinguin','brasa','mr','veggy','ecobras','org','superbom','texas',
              'geronimo','casa','rezende','sitio','miguel','cook','noiset','in','natu','wessel','atelie',
              'carone','sodebo']

# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9].+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')

# Função para retirar stop words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stop_words]
  new_text = TreebankWordDetokenizer().detokenize(stop)
  return new_text

# Eliminando as stop words
df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].apply(tokenize_df)

# Listando arquivo
dotz = df.query('DEPARTAMENTO == "CONGELADOS SALGADOS"')
dotz = dotz['DESC_AJUSTADA'] + '     |' + dotz['DEPARTAMENTO'] + '    |' + dotz['CATEGORIA']
dotz = sorted(list(dotz))

# Gerar um arquivo com a lista de palavras unicas
arquivo = open('dotz.txt', 'w')
arquivo.write(str(dotz))
arquivo.close()
'''

'\npontuacao = [\'.\',\',\',\'-\',\'+\',\':\',\';\',\'&\',\'+\',\'/\',\'!\',\'#\',\'%\',\'(\',\')\',\'  \']\nstop_words = [\'em\',\'sao\',\'ao\',\'de\',\'da\',\'do\',\'para\',\'c\',\'kg\',\'un\',\'ml\',\'pct\',\'und\',\'das\',\n              \'no\',\'ou\',\'sem\',\'gfa\',\'jg\',\'p/\',\'la\',\'1\',\'2\',\'3\',\'4\',\'5\',\'6\',\'7\',\'8\',\'9\',\'0\',\'a\',\n              \'b\',\'c\',\'d\',\'e\',\'f\',\'g\',\'h\',\'i\',\'j\',\'k\',\'l\',\'m\',\'n\',\'o\',\'p\',\'q\',\'r\',\'s\',\'t\',\'u\',\'v\',\n              \'x\',\'w\',\'y\',\'z\',\'kit\',\'po\',\'liq\',\'cr\',\'sh\',\'pt\',\'des\',\'imp\',\'lt\',\'vd\',\'az\',\'esm\',\n              \'cx\',\'bc\',\'lv\',\'ad\',\'gf\',\'mix\',\'rf\',\'fr\',\'bj\',\'int\',\'cd\',\'cz\',\'tp\',\'ap\',\'rs\',\'caixa\',\n              \'sao\',\'bom\',\'esp\',\'extra\',\'premium\',\'pto\',\'uso\',\'sol\',\'garrafa\',\'brasil\',\'pc\',\'conj\',\n              \'ph\',\'gold\',\'cj\',\'mais\',\'cor\',\'facil\',\'ice\',\'classic\',\'br\',\'vm\',\'tt\',\'em\